Script para descargar los avatares (fotos de perfil) de los seguidores de Twitter usando la API.

Preparación del entorno:

1) Guardar en el .env el ID del usuario del cual se quieren obtener los seguidores
2) Correr pastillas 1, 2 y 3
3) Si hay mas de 1000 seguidores, en pastilla 2 descomentar la linea de url += y poner el next_token de la ultima busqueda (de pastilla 3)

In [16]:
from dotenv import load_dotenv
import os
import requests
import json

# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Accede a los valores de las variables de entorno
user = os.getenv('USER')
token_bearer = os.getenv('TOKEN_BEARER')

In [17]:
# Función para obtener los datos de la API
def more_data(user, next_token):
    url = 'https://api.twitter.com/2/users/' + user + '/followers?user.fields=profile_image_url&max_results=100'
    
    if next_token != 0:
        url += '&pagination_token=' + next_token
    
    # Define los encabezados de la solicitud con el token de autenticación
    headers = {
        'Authorization': f'Bearer {token_bearer}'
    }
    
    # Realiza la solicitud HTTP con los encabezados
    response = requests.get(url, headers = headers)
    
    # Verifica el código de estado de la respuesta
    if response.status_code == 200:
        # Accede a los datos en formato JSON
        data = response.json()
                      
        print(f'Datos obtenidos. Código de estado: {response.status_code}')
        
        return data
   
    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        
        return

In [18]:
print('Ejecutando primera conexión')
data = more_data(user, 0)
seguidores = []

while True:
    if 'next_token' in data['meta']:
        respuesta = input('Hay más datos disponibles, desea descargarlos? (S/N): ')
        if respuesta.lower() == 's':
            seguidores.extend(data['data'])
            data = more_data(user, data['meta']['next_token'])
        else:
            break
    else:
        seguidores.extend(data['data'])
        print('Se descargaron todos los datos disponibles.')
        break

print('Total de seguidores obtenidos:', len(seguidores))

Ejecutando primera conexión
Datos obtenidos. Código de estado: 200
Se descargaron todos los datos disponibles.
Total de seguidores obtenidos: 43


In [2]:
# Define la url para obtener los datos
url = 'https://api.twitter.com/2/users/' + user + '/followers?user.fields=profile_image_url&max_results=100'

# Si es la primera busqueda, comentar la siguiente linea, sino poner el token de siguiente pagina obtenido en pastilla 3
# url += '&pagination_token=27OV50M0RU0HGZZZ'

# Define el nombre de archivo para guardar los datos JSON
nombre_archivo = 'datos.json'

# Define los encabezados de la solicitud con el token de autenticación
headers = {
    'Authorization': f'Bearer {token_bearer}'
}

# Realiza la solicitud HTTP con los encabezados
response = requests.get(url, headers = headers)

# Verifica el código de estado de la respuesta
if response.status_code == 200:
    # Accede a los datos en formato JSON
    data = response.json()
    
    # Guarda los datos en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
       json.dump(data, archivo)
    
    # print(f'Los datos se han guardado en el archivo "{nombre_archivo}"')
    print(f'Conexión establecida y datos obtenidos. Código de estado: {response.status_code}')
    
    if (data['meta']['next_token']):
        print('Hay mas datos para descargar... continuar S / N?')
    print(data['meta']['result_count'])
    
else:
    print(f'Error en la solicitud. Código de estado: {response.status_code}')

Conexión establecida y datos obtenidos. Código de estado: 200
43


In [30]:
# Guardar el array en un archivo JSON separado
array_data = data['data']

users_file = 'users001.json'

with open(users_file, 'w') as file:
    json.dump(array_data, file)

print(f'Los datos se guardaron en el archivo {users_file}')

# Imprimir los valores de 'result' y 'next_token'
result_value = data['meta']['result_count']
next_token_value = data['meta']['next_token']

print(f'Resultados: {result_value}')
print(f'next_token: {next_token_value}')

Los datos se guardaron en el archivo users001.json
Resultados: 100
next_token: 27OV50M0RU0HGZZZ


# Descarga de imágenes de Twitter

In [2]:
# Constantes MODIFICAR
# RUTA: carpeta donde se van a guardar las imagenes descargadas
# ARCHIVO_HAR: archivo HAR descargado de Twitter
# ARCHIVO_JSON: archivo temporal que contiene solo los nombres de usuarios y links a fotos
# CADENA: cadena de texto que se usa para encontrar los fragmentos donde estan los nombres de usuarios y links a fotos

RUTA = 'fotos/'
ARCHIVO_HAR = 'twitter.com.har'
ARCHIVO_JSON = 'salida.json'
CADENA = r'"text": "{\"data\":{\"user\":{\"result\":{\"__typename\":\"User\",\"timeline\":{\"timeline\":{\"instructions\":[{\"type\":\"TimelineAddEntries\",\"entries\":['

In [3]:
# Importar librerías

import re
import pandas as pd
import json
import os.path
from os import path
import time
import numpy as np
import urllib.request, urllib.error
from IPython.display import clear_output
import shutil

from tqdm import tqdm

In [4]:
# Crea el dataframe 1 con los datos obtenidos del HAR y exporta todo como un nuevo archivo JSON
# Esto es necesario porque los datos necesarios estan dentro de un par llave/valor como texto y formateado JSON
# Es decir, es más fácil primero obtener los textos y luego parsear esos textos

# Define la estructura del dataframe 1 con una sola columna
columnas = ['fila']
df1 = pd.DataFrame(columns = columnas)

# Abre el archivo HAR y lee línea por línea, si en la linea encuentra la 'CADENA', la porcesa y la guarda en el dataframe 1
with open(ARCHIVO_HAR, 'r', encoding = 'utf-8') as file:
  for line in file:
    if CADENA in line:
      line1 = line.replace(CADENA, '')
      line2 = line1.replace(r']}]}}}}}}"', '')
      line3 = line2.replace(r'\\', r'\0x0A06eaE4129d2C45f36A074A50A3AaF4460051D0')  # como la barra invertida no puede ser el ultimo caracter de una cadena
      line4 = line3.replace(r'0x0A06eaE4129d2C45f36A074A50A3AaF4460051D0', '')      # pongo mi dirección de ethereum como texto random
      df1.loc[len(df1.index)] = line4.replace(r'\"', '"')
  print('Procesadas:',df1.shape[0],'cadenas de texto')

# Cuenta cuántas líneas se generaron (-1 porque el dataframe cuenta desde cero)
# row_count = df1.shape[0] - 1 # PASADO AL IF rec_index a continuacion

# Crea el archivo JSON y guarda el dataframe 1 como JSON, agerga comas entre lineas del dataframe 1 y un cierre al final
with tqdm(total=len(df1.index)) as pbar:
  with open(ARCHIVO_JSON, 'w', encoding = 'utf-8') as f:
    f.write('{"entries":[')
    for rec_index, rec in df1.iterrows():
      if rec_index != (df1.shape[0] - 1):
        f.write(rec['fila'] + ',')
      else:
        f.write(rec['fila'])
      pbar.update(1)
    f.write(']}')

Procesadas: 127 cadenas de texto


100%|██████████| 127/127 [00:00<00:00, 1483.03it/s]


In [5]:
# Crea el dataframe 2 con nombre y link de imagenes de los seguidores

# Define la estructura del dataframe con dos columnas
columnas = ['error','nombre', 'imagen']
df2 = pd.DataFrame(columns = columnas)

# Abre el archivo JSON creado en la celda anterior
with open(ARCHIVO_JSON, encoding = 'utf-8') as f:
  data = json.load(f)

# Extrae nombre de usuario y link a la imagen y lo mete en el dataframe
with tqdm(total=len(data['entries'])) as pbar:
  for i in range(len(data['entries'])):
    if 'itemContent' in data['entries'][i]['content']:
      df2.loc[len(df2.index)] = [0, data['entries'][i]['content']['itemContent']['user_results']['result']['legacy']['screen_name'], data['entries'][i]['content']['itemContent']['user_results']['result']['legacy']['profile_image_url_https']]
    pbar.update(1)

100%|██████████| 2793/2793 [00:02<00:00, 1337.20it/s]


In [5]:
df2

,error,nombre,imagen
0,0,totiwin,https://pbs.twimg.com/profile_images/137795405...
1,0,CorzoFillmore,https://pbs.twimg.com/profile_images/148166911...
2,0,CEDICE,https://pbs.twimg.com/profile_images/134900024...
3,0,NicolasAroca,https://pbs.twimg.com/profile_images/123893933...
4,0,678594ad162d427,https://pbs.twimg.com/profile_images/148324713...
...,...,...,...
2534,0,adormilanesa,https://pbs.twimg.com/profile_images/126562210...
2535,0,katz00T,https://pbs.twimg.com/profile_images/137675917...
2536,0,vinoandpritty,https://pbs.twimg.com/profile_images/150088928...
2537,0,pablosc88,https://pbs.twimg.com/profile_images/130444971...


In [42]:
# Crea la carpeta donde se van a descargar las imagenes

if path.exists(RUTA) == False:
  os.mkdir(RUTA)

In [6]:
# Descarga imagenes en base al dataframe 2

# Funcion que descarga cada imagen de Twitter
# i: numero de imagen / url: direccion web / ruta: carpeta donde se guardan las imagenes
def url_to_imagen(i, url, ruta):
  # guarda las extensiones jpg o jpeg como jpg
  if url[-3:] == 'jpg' or url[-4:] == 'jpeg' or url[-3:] == 'JPG' or url[-4:] == 'JPEG':
    archivo = '{}.jpg'.format(str(i).zfill(6))
    guardado = '{}{}'.format(ruta, archivo)
  # si es png lo guarda como png
  elif url[-3:] == 'png' or url[-3:] == 'PNG':
    archivo = '{}.png'.format(str(i).zfill(6))
    guardado = '{}{}'.format(ruta, archivo)
  # si es cualquier otra extension (poco probable) o no tiene link, usa el avatar por defecto
  else:
    archivo = '{}.png'.format(str(i).zfill(6))
    url = 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png'
    guardado = '{}{}'.format(ruta, archivo)
  # intenta descarga el archivo, si logra descargarlo marca el error como CERO
  try:
    con = urllib.request.urlretrieve(url, guardado)
    df2.at[i, 'error'] = 0
  # si hubo error al descargar, suma 1 al registro de error de esa imagen
  except urllib.error.HTTPError as err:
    df2.at[i, 'error'] = df2.at[i, 'error'] + 1
  # minima demora para no saturar la conexion (probar el valor mas efectivo)
  time.sleep(0.15)
  return None

# Guarda imagenes en la carpeta iterando por el dataframe 2
# (iterar no es lo mejor en pandas, pero son pocas operaciones e igualmente deben tener algo de demora)
with tqdm(total=len(df2.index)) as pbar:
  for i, row in df2.iterrows():
    #clear_output(wait = True)
    # en cada iteracion llama a la funcion que descarga la imagen
    url_to_imagen(i, row['imagen'], RUTA)
    # muestra el progreso de la operación
    #avance = np.round(i/len(df2) * 100, 2)
    #print('Progreso:',i,'de',len(df2),'-',avance,'%')
    #if i == 200:
    #  break
    pbar.update(1)

# Suma los errores obtenidos
errores = df2.loc[df2['error'] > 0].index[0]

# Avisa si huno errores al descargar imagenes
if errores > 0:
  print ('\n')
  print ('Errores:',errores)
  print (df2.loc[df2['error'] > 0])
else:
  print ('Todas las imagenes se descargaron sin errores')

100%|██████████| 2539/2539 [14:20<00:00,  2.95it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
df2.loc[df2['nombre'] == 'marcaemi'].index[0]

191

In [ ]:
# Descarga de imagenes que no se pudieron bajar por error
# Escribir codigo que establezca una cantidad de veces a probar, y que itere por el dataframe 2 en las filas con error > a cero
# luego de todos los intentos, a las que no se pudieron descargar se les asigna la imagen por defecto
url_to_imagen(2459, 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', RUTA)


In [ ]:
# -----------------------------------------------------------------------------------------------------------
# IMPORTANTE: Esto no es necesario, es para borrar la carpeta imagenes completa por si hace falta probar algo

shutil.rmtree('imagenes')
# -----------------------------------------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------------------------------------
# IMPORTANTE: Esto no es necesario ejecutarlo, es por si hace falta descargar las imagenes, es mas rapido en un solo archivo comprimido grande.
# Comprime carpeta con imagenes y guardar el archivo para poder descargar

shutil.make_archive('fotos', 'zip', RUTA)
# -----------------------------------------------------------------------------------------------------------

# Creación de mosaico de imágenes

Origen: https://towardsdatascience.com/how-to-create-a-photo-mosaic-in-python-45c94f6e8308

Importamos todas las librearías necesarias

In [2]:
# Importar librerías

import glob                   # para trabajar con carpetas y archivos
import os.path                # idem
from PIL import Image         # para imágenes
#from scipy import spatial     # ¿NO SE USA?
import numpy as np            # para funciones matemáticas
from numpy.ma.core import mean  # para promedios
import pandas as pd           # para dataframes
import time                   # para registro de tiempo
from tqdm import tqdm         # para barras de progreso
#from matplotlib import pyplot as plt
#from matplotlib import image

In [ ]:
from IPython.display import clear_output  # CAPAZ QUE NO HACE FALTA


**Seteamos las rutas de los archivos y algunos datos globales.**

archivo_imagen_base: es la ubicación y nombre de la imagen base (o de fondo) sobre la cual se realizará el mosaico.

archivo_imagen_final: es la ubicación y nombre de la imagen final (trabajo terminado)

ruta_imagenes_teselas: es la carpeta donde estan las imágenes que componen el mosaico.

imagenes_finales: es la carpeta donde se guardaran las imágenes procesadas de las teselas (por cambio de formato por ejemplo)

medida_tesela: es el tamaño que va a tener cada "tesela" del mosaico.

In [3]:
# Seteo de ubicaciones, archivos y datos

archivo_imagen_base = 'entrada.jpg'
archivo_imagen_final = 'final.jpg'
#archivos_teselas_originales = "/content/drive/MyDrive/Colab Notebooks/fotos/*"
archivos_teselas_originales = "fotos"
archivos_teselas_finales = 'imagenes'
medida_tesela = (48, 48)

**Formateo imagen base**

Pixelamos la imagen base de manera tal que en su ancho/alto original entren "teselas" del tamaño seteado previamente.

In [4]:
# Pixelate (resize) main photo

imagen_base = Image.open(archivo_imagen_base)

X = int(np.round(imagen_base.size[0] / medida_tesela[0]))
Y = int(np.round(imagen_base.size[1] / medida_tesela[1]))

imagen_pixelada = imagen_base.resize((X, Y))

# OJO CON ESTA CELDA: VER ESTA DESCRIPCION
# To pixelate the main photo, we will scale it down using Pillow’s resize
# method. We will resize it so that every pixel in the resized photos is
# exactly the size of a tile in the original photo.

**Conversión de archivos**

En la siguiente celda convertimos en JPG de 24 bits a los archivos PNG y a los JPG de cualquier otra profundidad de color.

El FOR recorre todos los archivos, abre cada uno, lo convierte y lo guarda en una nueva carpeta.

(probar si funciona con otros formatos, como GIF, JPEG, BMP etc)

In [11]:
# Convierte las imagenes a RGB y las guarda en la nueva carpeta

with tqdm(total = len(glob.glob(archivos_teselas_originales + r'\\*'))) as pbar:
  for archivo in glob.glob(archivos_teselas_originales + r'\\*'):
    with Image.open(archivo) as imagen:
      imagen_rgb = imagen.convert('RGB')
      imagen.mode = 'RGB'
      imagen_rgb.save(archivos_teselas_finales + r'\\' + os.path.basename(archivo[:-3]) + 'jpg')
    pbar.update(1)

100%|██████████| 2559/2559 [00:41<00:00, 61.02it/s]


**Carga ubicación de teselas modificadas**

En esta celda se carga la ubicación de todas las teselas nuevas en rutas_teselas.

In [5]:
# Obtener ubicación imágenes

rutas_teselas = []
for archivo in glob.glob(archivos_teselas_finales + r'\\*'):
	rutas_teselas.append(archivo)
 # revisar que contiene tile_paths para ver como renombrarla y si esta bien el titulo de este codigo

**Carga de imágenes en memoria**

Cargamos todas las imágenes en memoria, en la ¿variable? tiles

In [6]:
# Carga y redimensiona imagenes solo hasta la cantidad de teselas necesarias

teselas = []
for path in rutas_teselas:
	tesela = Image.open(path)
	#tesela = tesela.resize(medida_tesela)
	teselas.append(tesela)
	if len(teselas) == (X*Y):
		break
 # idem codigo anterior, ver si path es ruta o que cosa

**Color predominante de cada imagen**

Creamos un dataframe vacío y algunas variables.

Ahora recorremos todas las imágenes cargadas en tile y por cada una obtenemos el color promedio, el resultado lo almacenamos en un dataframe donde además agregamos otras variables:
- False: para indicar que la imagen aun no se usó en el mosaico final.
- mean_color: es el color promedio
- x: con valor por defecto CERO, indica eje x de posición en mosaico final.
- y: con valor por defecto CERO, indica eje y de posición en mosaico final.

In [7]:
# Calculate dominant color

columnas = ['usado','color','x','y']
df3 = pd.DataFrame(columns = columnas)

coloress = []
n = 0

with tqdm(total = len(teselas)) as pbar:
  for tesela in teselas:
    n = n + 1
    color_promedio = np.array(tesela).mean(axis=0).mean(axis=0)
    coloress.append(color_promedio)
    df3.loc[len(df3.index)] = [False, color_promedio, 0, 0]
    pbar.update(1)

100%|██████████| 2209/2209 [00:02<00:00, 830.44it/s]


In [8]:
# Guarda el dataframe en un archivo OPCIONAL

df3.to_pickle('df3.pkl')

**Función para encontrar distancias**

Los colores RGB pueden tomarse como unidades de un espacio tridimensional, de esta forma podemos medir la distancia entre colores (o puntos en el espacio)

In [9]:
def buscar_distancia(color1,color2) :
    [x1,y1,z1] = color1  # first coordinates
    [x2,y2,z2] = color2  # second coordinates

    return (((x2-x1)**2)+((y2-y1)**2)+((z2-z1)**2))**(1/2)

**Carga del dataframe de un archivo**

In [10]:
# Recupera el dataframe del archivo guardado (es para evitar hacer todo el proceso anterior de nuevo)

df4 = pd.read_pickle('df3.pkl')

**Generación de imagen final (patrón radial)**

En esta celda recorremos la imagen final en un patrón espiralado, partiendo desde el centro. Para que esto funcione bien la imagen base tiene que ser cuadrada y tiene que tener un número impar de píxeles en X y en Y, ya que el centro de la imagen será la coordenada 0,0

In [13]:
if X == Y:
  if (X % 2) != 0:
    x = y = 0
    dx = 0
    dy = -1
    with tqdm(total = X*Y) as pbar:
      for i in range(max(X, Y)**2):
        if (-X/2 < x <= X/2) and (-Y/2 < y <= Y/2):
          real_x = X/2 - (0.5) + x
          real_y = Y/2 - (0.5) + y
          pixel = imagen_pixelada.getpixel((real_x, real_y))
          distancia = 256
          id_pixel = 0
          for n, row in df3.iterrows():
            if buscar_distancia(pixel, row['color']) < distancia and row['usado'] == False:
              distancia = buscar_distancia(pixel, row['color'])
              id_pixel = n
          df3.at[id_pixel, 'usado'] = True
          df3.at[id_pixel, 'x'] = real_x
          df3.at[id_pixel, 'y'] = real_y
        if x == y or (x < 0 and x == -y) or (x > 0 and x == 1-y):
          dx, dy = -dy, dx
        x, y = x+dx, y+dy
        pbar.update(1)
  else:
    print ('No se puede usar este patrón radial en una imagen con una resolución par de pixeles. La imagen mide',X,'x',Y,'px, pero X e Y deben ser impares en la imagen base.')
else:
  print ('No se puede usar este patrón radial en una imagen no cuadrada, X e Y son distintos en la imagen base. La imagen mide',X,'x',Y,'px.')


100%|██████████| 2209/2209 [07:14<00:00,  5.09it/s]


**Generación de imagen final (patrón al azar)**

Genera la imagen eligiendo pixeles al azar sin repetirlos

In [39]:
# Genera la imagen al azar
import random
import math

# crea una lista con numeros random X*Y que no se repiten
aleatorios = random.sample(range(1, (X*Y)+1), (X*Y))
id_pixel = 0
#print(aleatorios)

with tqdm(total = X*Y) as pbar:
  for i in range(len(aleatorios)):
    # obtener la ubicacion del random como X Y
    # y = si residuo o modulo del pixel buscado es = 0
    #   entero de la division entre el pixel buscado y el ancho de la imagen
    #   sino (entero de la division entre el pixel buscado y el ancho de la imagen) + 1 
    if (aleatorios[i] % X == 0):
      y = math.trunc(aleatorios[i] / X) - 1
    else:
      y = math.trunc(aleatorios[i] / X)
    # x = n pixel - ( ancho * (y - 1))
    x = aleatorios[i] - (X * (y)) - 1

    pixel = imagen_pixelada.getpixel((x, y))
    distancia = 256
    for n, row in df4.iterrows():
        distancia_temporal = buscar_distancia(pixel, row['color'])
        if distancia_temporal < distancia and row['usado'] == False:
            distancia = distancia_temporal
            id_pixel = n
            df4.at[id_pixel, 'usado'] = True
            df4.at[id_pixel, 'x'] = x
            df4.at[id_pixel, 'y'] = y
    pbar.update(1)

100%|██████████| 625/625 [00:31<00:00, 20.07it/s]


**Generación de imagen final (patrón X Y)**

Genera la imagen en base a un patrón de filas y columnas, comienza por la esquina superior izquierda y hace las filas de izquierda a derecha, y de arriba hacia abajo

In [210]:
# Genera la imagen en grilla X e Y, recorriendo por filas

id_pixel = 0

with tqdm(total = X*Y) as pbar:
  for y in range(Y):
    for x in range(X):
      distancia = 256
      pixel = imagen_pixelada.getpixel((x, y))
      for n, row in df4.iterrows():
        distancia_temporal = buscar_distancia(pixel, row['color'])
        if distancia_temporal < distancia and row['usado'] == False:
          distancia = distancia_temporal
          id_pixel = n
      df4.at[id_pixel, 'usado'] = True
      df4.at[id_pixel, 'x'] = x
      df4.at[id_pixel, 'y'] = y
      pbar.update(1)

100%|██████████| 2209/2209 [06:36<00:00,  5.57it/s]


In [17]:
# Prueba recorriendo primero el dataframe, es lo ideal

id_pixel = 0

with tqdm(total = len(df4)) as pbar:
    for n, row in df4.iterrows():
        distancia = 256
        for y in range(Y):
            for x in range(X):
                pixel = imagen_pixelada.getpixel((x, y))
                distancia_temporal = buscar_distancia(pixel, row['color'])
                if distancia_temporal < distancia and row['usado'] == False:
                    distancia = distancia_temporal
                    id_pixel = n
        pbar.update(1)
        df4.at[id_pixel, 'usado'] = True
        df4.at[id_pixel, 'x'] = x
        df4.at[id_pixel, 'y'] = y
    

103823it [00:39, 2605.25it/s]


In [18]:
df4

,usado,color,x,y
0,True,"[152.74305555555554, 123.53081597222221, 111.7...",46,46
1,True,"[139.56814236111111, 141.52864583333334, 143.8...",46,46
2,True,"[134.70963541666666, 147.3394097222222, 163.00...",46,46
3,True,"[2.793836805555554, 50.80729166666665, 113.749...",46,46
4,True,"[135.94270833333334, 114.39713541666669, 114.6...",46,46
...,...,...,...,...
2204,True,"[142.24739583333334, 139.6684027777778, 138.58...",46,46
2205,True,"[115.48741319444441, 85.68012152777779, 73.781...",46,46
2206,True,"[58.95789930555554, 68.87586805555556, 59.5520...",46,46
2207,True,"[70.7517361111111, 69.52083333333334, 67.69574...",46,46


**Creación de archivo final**

En base a las coordenadas guardadas en el dataframe, generamos el archivo final pegando cada "tesela" en su correspondiente ubicación.

In [15]:
# Genera la imagen final en base al dataframe
salida = Image.new('RGB', imagen_base.size)

# Draw tiles
with tqdm(total = X*Y) as pbar:
	for n, row in df4.iterrows():
		x = medida_tesela[0] * row['x']
		y = medida_tesela[1] * row['y']
		salida.paste(teselas[n], (x, y))
		pbar.update(1)

# Save output
salida.save(archivo_imagen_final)

100%|██████████| 2209/2209 [00:00<00:00, 27644.08it/s]


In [ ]:
salida

# Código descartado

In [ ]:
# Create an output image
salida = Image.new('RGB', imagen_base.size)

# Draw tiles
with tqdm(total = X*Y) as pbar:
	for i in range(X):
		for j in range(Y):
			# Offset of tile
			x = i * medida_tesela[0]
			y = j * medida_tesela[1]
			# Index of tile
			index = df4.loc[((df4['x'] == i) & (df4['y'] == j))].index[0]
			# Draw tile
			salida.paste(teselas[index], (x, y))
			pbar.update(1)

# Save output
salida.save(archivo_imagen_final)

# Creación de mosaico opcion 2

In [ ]:
# Importar módulos

from PIL import Image, ImageOps
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image
import glob, os
import math
from scipy import spatial
import random 

In [ ]:
# Función obtiene un array numpy de una imagen

def load_image(source : str) -> np.ndarray:
  # Opens an image from specified source and returns a numpy array with image rgb data
  with Image.open(source) as im:
    im_arr = np.asarray(im)
  return im_arr

In [ ]:
# Carga la imagen base

plt.figure(figsize=(8,8))
face_im_arr = load_image('input.jpg')
face_image = Image.fromarray(face_im_arr)
plt.imshow(face_image)
plt.show()

In [ ]:
# Guarda el ancho y el alto de la imagen base

width = face_im_arr.shape[0]
height = face_im_arr.shape[1]

In [ ]:
# Establece la resolución final, la cual coincidirá con la cantidad de 
# imagenes pequeñas que forma el mosaico, debe ser 1 menos que el tamaño buscado
# Por ejemplo, si queremos de 50 x 50, poner 49 x 49
# DEBE SER CUADRADO DE FILAS Y COLUMNAS IMPARES

target_res = (58, 58) ## Number of images to make up final image (width, height)

In [ ]:
# Crea un modelo para el mosaico

plt.figure(figsize=(8,8))
mos_template = face_im_arr[::(height//target_res[0]),::(height//target_res[1])]
plt.imshow(Image.fromarray(mos_template))
plt.show()

In [ ]:
# Cantidad de imagenes para llenar el mosaico, no hace falta ejecutar esto

mos_template[:,:, -1].size

In [ ]:
# Crea una lista de todas las imagenes como arrays np

images = []

for file in glob.glob('imagenes/*'):
  im = load_image(file)
  images.append(im)

In [ ]:
# Establece el tamaño de las imagenes del mosaico, y las redimensiona

mosaic_size = (48, 48) ## Defines size of each mosiac image
images = [resize_image(Image.fromarray(i), mosaic_size) for i in images]

In [ ]:
# Cantidad de imagenes en la lista, no hace falta ejecutar esto

len(images)

In [ ]:
# Convierte la lista en un array np

images_array = np.asarray(images)
images_array.shape

In [ ]:
# Obtiene el RGB promedio de cada imagen

image_values = np.apply_over_axes(np.mean, images_array, [1,2]).reshape(len(images),3)
image_values

In [ ]:
# Establece el espacio KDTree

tree = spatial.KDTree(image_values)

In [ ]:
# Guarda los indices para las mejores coincidencias
# Loop through each pixel in the low-res reference image and find the index of the image that has the closest mean colour.<br>
# **The 'match' variable returns a list of the closest 40 matches and one is randomly chosen - this is to reduce repeat images for similar colours that are close to each other. This number would need to be smaller than the number of images available**

image_idx = np.zeros(target_res, dtype=np.uint32)

for i in range(target_res[0]):
  for j in range(target_res[1]):
        
    template = mos_template[i, j]

    match = tree.query(template, k=40)
    pick = random.randint(0, 39)
    image_idx[i, j] = match[1][pick]

In [ ]:
# Loopea por las mejores coincidencias, obtiene la imagen y la agrega al modelo

canvas = Image.new('RGB', (mosaic_size[0]*target_res[0], mosaic_size[1]*target_res[1]))

for i in range(target_res[0]):
  for j in range(target_res[1]):
    arr = images[image_idx[j, i]]
    x, y = i*mosaic_size[0], j*mosaic_size[1]
    im = Image.fromarray(arr)
    canvas.paste(im, (x,y))

In [ ]:
# Muestra la imagen final

canvas